In [1]:
# Upgrade pip and install ffmpeg
!pip install --upgrade pip
!apt-get install ffmpeg -y

# Install torch and Whisper
!pip install torch --quiet
!pip install git+https://github.com/openai/whisper.git --quiet

# Install other dependencies
!pip install vosk streamlit pyngrok jiwer soundfile groq --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'srt' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 's

In [2]:
import os
import torch
import whisper
import soundfile as sf
from vosk import Model, KaldiRecognizer
from groq import Groq
from jiwer import wer
import json


In [3]:
# Set your Groq API key here
# Replace "gsk_your_actual_api_key_here" with your actual key
os.environ["GROQ_API_KEY"] = "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"


In [4]:
os.makedirs("meeting_pipeline", exist_ok=True)


In [5]:
whisper_model = whisper.load_model("small")


100%|███████████████████████████████████████| 461M/461M [00:12<00:00, 39.1MiB/s]


In [6]:
# Download a small Vosk English model if not present
if not os.path.exists("vosk-model-small-en-us-0.15"):
    !wget -q https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip
    !unzip -q vosk-model-small-en-us-0.15.zip

vosk_model = Model("vosk-model-small-en-us-0.15")


In [7]:
# Initialize Groq client
groq_client = Groq(api_key=os.getenv("GROQ_API_KEY"))

# Summarization function using Groq 'compound' model
def summarize_with_groq(text):
    response = groq_client.chat.completions.create(
        model="groq/compound",  # Model your key has access to
        messages=[
            {"role": "system", "content": "You are a professional meeting summarizer. Summarize the transcript clearly and concisely."},
            {"role": "user", "content": f"Meeting Transcript:\n{text}"}
        ],
        temperature=0.5,
        max_tokens=300
    )
    return response.choices[0].message.content.strip()


In [8]:
# Whisper transcription
def transcribe_audio(file_path):
    result = whisper_model.transcribe(file_path)
    return result["text"]

# Vosk diarization
def diarize_audio(file_path):
    import wave
    wf = wave.open(file_path, "rb")
    rec = KaldiRecognizer(vosk_model, wf.getframerate())
    segments = []
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            res = json.loads(rec.Result())
            if 'text' in res:
                segments.append(res['text'])
    final_res = json.loads(rec.FinalResult())
    if 'text' in final_res:
        segments.append(final_res['text'])
    return segments

# Summarization using Groq
def summarize_text(text):
    return summarize_with_groq(text)


In [9]:
app_code = """\
import streamlit as st
import whisper
from vosk import Model, KaldiRecognizer
import wave
import json
from groq import Groq
import os

@st.cache_resource
def load_models():
    whisper_model = whisper.load_model("small")
    vosk_model = Model("vosk-model-small-en-us-0.15")
    groq_client = Groq(api_key=os.getenv("GROQ_API_KEY"))
    return whisper_model, vosk_model, groq_client

whisper_model, vosk_model, groq_client = load_models()

st.title("Meeting Transcription + Diarization + Summarization (GroqAI)")

uploaded_file = st.file_uploader("Upload a meeting .wav file", type=["wav"])

def summarize_with_groq(client, text):
    response = client.chat.completions.create(
        model="groq/compound",  # Accessible model
        messages=[
            {"role": "system", "content": "You are a professional meeting summarizer. Summarize the transcript clearly and concisely."},
            {"role": "user", "content": f"Meeting Transcript:\\n{text}"}
        ],
        temperature=0.5,
        max_tokens=300
    )
    return response.choices[0].message.content.strip()

if uploaded_file is not None:
    with open("temp.wav", "wb") as f:
        f.write(uploaded_file.read())

    st.info("Transcribing...")
    transcription = whisper_model.transcribe("temp.wav")["text"]
    st.success("Transcription complete!")
    st.text_area("Transcription", transcription, height=200)

    st.info("Performing diarization...")
    wf = wave.open("temp.wav", "rb")
    rec = KaldiRecognizer(vosk_model, wf.getframerate())
    segments = []
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            res = json.loads(rec.Result())
            if 'text' in res:
                segments.append(res['text'])
    final_res = json.loads(rec.FinalResult())
    if 'text' in final_res:
        segments.append(final_res['text'])
    st.success("Diarization complete!")
    st.text_area("Diarized Transcript", "\\n".join(segments), height=200)

    st.info("Summarizing with GroqAI...")
    summary = summarize_with_groq(groq_client, transcription)
    st.success("Summary complete!")
    st.text_area("Meeting Summary", summary, height=150)
"""

with open("/content/meeting_pipeline/app.py", "w") as f:
    f.write(app_code)


In [10]:
!pip install pyngrok --quiet


In [11]:
from pyngrok import ngrok

# Replace with your own ngrok auth token
ngrok.set_auth_token("33VBAydDytS0a0YnyvXwwzws4pr_5g5YRpRzom9RcK1STJNU9")


In [12]:
from pyngrok import ngrok

# Kill all existing tunnels
ngrok.kill()

# Then start a new tunnel
public_url = ngrok.connect(8501).public_url
print("Streamlit public URL:", public_url)


Streamlit public URL: https://emboly-urgent-kristine.ngrok-free.dev


In [13]:
!ls /content/meeting_pipeline/app.py


/content/meeting_pipeline/app.py


In [14]:
!pkill -f streamlit


In [15]:
!nohup streamlit run /content/meeting_pipeline/app.py --server.port 8501 > /content/meeting_pipeline/streamlit.log 2>&1 &


In [16]:
import time
time.sleep(10)  # Wait 10 seconds for the server to initialize


In [17]:
!tail -n 20 /content/meeting_pipeline/streamlit.log





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.199.99:8501



In [18]:
from pyngrok import ngrok

# Kill any old tunnels
ngrok.kill()

# Start new tunnel on the port Streamlit is running
public_url = ngrok.connect(8501).public_url
print("Streamlit public URL:", public_url)


Streamlit public URL: https://emboly-urgent-kristine.ngrok-free.dev
